In [ ]:
import os
import sys
import json
import tempfile
import pandas as pd
import numpy as np

import io

from CoolProp.CoolProp import PropsSI

# shortcuts
cwd = os.getcwd()
join = os.path.join
norm = os.path.normpath

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = [15, 9]
plt.rcParams['font.size'] = 14

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [ ]:
def add_delimiters(fpath, delimiter=','):
    
    s_data = ''
    max_num_delimiters = 0
    
    with open(fpath, 'r') as f:
        for line in f:
            s_data += line
            delimiter_count = line.count(delimiter)
            if delimiter_count > max_num_delimiters:
                max_num_delimiters = delimiter_count
    
    s_delimiters = delimiter * max_num_delimiters + '\n'
    
    return io.StringIO(s_delimiters + s_data)

In [ ]:
def load_data(d_files, add_delimiters_to_header=False):
    dfs = []
    
    for key, file in sorted(d_files.items()):
        
        if add_delimiters_to_header:
            d = pd.read_csv(add_delimiters(file), index_col=0)
        else:
            d = pd.read_csv(file, index_col=0, header=None)
            d.drop(labels=[1], axis=1, inplace=True)
            
        d.index.name = 'time stamp'
        
        d.head()
            
        d['data'] = key
        dfs.append(d)
        
    return pd.concat(dfs).reset_index().set_index(['time stamp', 'data'])

In [ ]:
f_dyna = {'Load': 'loads_dynamic.csv',
          'g-val': 'g_dynamic.csv',
          'Width': 'widths_dynamic.csv'}

df_dynamic = load_data(f_dyna)
df_dynamic.head()


In [ ]:
f_stat = {'Load': 'loads_static.csv',
          'g-val': 'g_static.csv',
          'Width': 'widths_static.csv'}

df_static = load_data(f_stat, True)
df_static.head()

In [ ]:
f_none = {'Load': 'loads_none.csv',
          'g-val': 'g_none.csv',
          'Width': 'widths_none.csv'}

df_none = load_data(f_none, True)
df_none.head()

In [ ]:
df_none.loc[240, 'Load'][0]

In [ ]:
list(df_none.loc[240, 'Load'].dropna())

In [ ]:
list(df_none.loc[240, 'Width'].dropna())

In [ ]:
list(df_none.loc[240, 'g-val'].dropna())

In [ ]:
c_0 = 1 / (2 * np.pi * 2.7)

In [ ]:
def calc_history_temp_rise(df, timestep):

    temp_rise_sum = 0

    def get_load(energy, width):
        return energy / width
    
    def calc_temp_rise(energy_i, 
                       energy_i_minus_1, 
                       width_i,
                       width_i_minus_1,
                       g):
        load_i = get_load(energy_i, width_i)
        load_i_minus_1 = get_load(energy_i_minus_1, width_i_minus_1)
        return (load_i - load_i_minus_1) * g * c_0

    num = df.loc[timestep, 'Load'].count()
    
    loads = list(df.loc[timestep, 'Load'].dropna())
    widths = list(df.loc[timestep, 'Width'].dropna())
    gs = list(df.loc[timestep, 'g-val'].dropna())
    
    for i in range(num - 1):
        energy_i = loads[i]
        energy_i_minus_1 = loads[i + 1]
        width_i = widths[i]
        width_i_minus_1 = widths[i + 1]
        g = gs[i + 1]
        
        temp_rise_sum += calc_temp_rise(energy_i,
                                        energy_i_minus_1,
                                        width_i,
                                        width_i_minus_1,
                                        g)

    energy = loads[-1]
    width = widths[-1]
    g = gs[-1]
    
    temp_rise_sum += get_load(energy, width) * g * c_0
    
    return temp_rise_sum

In [ ]:
calc_history_temp_rise(df_none, 120)

In [ ]:
t_g = 16.1
r_b = 0.1621

In [ ]:
def calc_MFT(df, timestep):
    energy = df.loc[timestep, 'Load'][0]
    width = df.loc[timestep, 'Width'][0]
    
    load = energy / width
    
    return t_g + calc_history_temp_rise(df, 120)